# Deep MNIST for Experts

这一节，我们学习使用Tensorflow来创建一个深度卷积MNIST分类器。

## Setup

在创建模型之前，我们先读取MNIST数据集，启动Tensorflow session。

### Load MNIST Data

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### 启动TensorFlow InteractiveSessioni

TensorFlow依赖C++后台进行计算。如何连接后台呢？方法是使用session。Tensorflow程序的步骤通常分两步：先创建计算图，然后在session中启动程序。

这里，我们使用更便捷的`InteractiveSession`类，它的优点是允许用户交错操作来构造一个计算图，而不是先构建好整个的计算图再启动session进行计算。InteractiveSession在IPython环境下使用非常便捷。

In [14]:
import tensorflow as tf
sess = tf.InteractiveSession()

## 创建一个Softmax Regression Model


我们先创建一个只有一个线性层的softmax regression model，接下来再创建复杂的带有多层卷积网络的Softmax regression。

### placeholder

我们先创建输入图像节点和输出目标类别节点。

In [15]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

虽然`shape`参数是可选的，但如果提供此参数，可以帮助Tensorflow自动检测代码中存在的tensor的shape不一致的bug。

### Variable

我们再来定义模型中的权重参数`W`和偏置`bias`。

In [16]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Variables在使用之前，必须在session中进行初始化。

In [17]:
sess.run(tf.global_variables_initializer())

### 预测类别和损失函数

现在实现我们的Softmax regression model。模型只有一行代码！

In [28]:
y = tf.matmul(x, W) + b

实现交叉熵损失函数，调用`tf.nn.softmax_cross_entropy_with_logits`

In [29]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

### 训练模型

In [30]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [31]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

每个batch大小是100个训练点。

## 评估模型

In [32]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

In [33]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [34]:
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9133


把模型代码整到一起：

In [36]:
import tensorflow as tf

sess = tf.InteractiveSession()

# 数据集
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

# 模型参数
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 初始化Variable
sess.run(tf.global_variables_initializer())

# 模型，只有一行代码
y = tf.matmul(x,W) + b

# 损失函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# 优化算法
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x:batch[0], y_:batch[1]})

# 评估模型
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels}))

0.9144


##  创建多层卷积神经网络

对于MNIST数据集来说，92%的准确率很一般。这一节我们创建一个卷积神经网络，准确率能达到99%。

### 权重初始化

神经网络含有许多权重参数和偏置参数。在初始化权重参数时通常用带有噪音的高斯分布，由于我们用ReLU神经元，所以初始化时要把标准差向正数方向偏移一下来避免“dead neurons”。

我们创建两个方法来避免重复性的代码：

In [39]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)




### 卷积和pooling

Tensorflow提供了许多卷积和pooling操作。

In [42]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2_plus_2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

### 第一个卷积层

我们来实现第一个卷积层，patch(local receptive field)大小是5*5，32个feature map，因此权重张量的shape是`[5, 5, 1, 32]`，前两个维度表示patch的大小，接下来是输入channel的个数，输出channel的个数。每个输出channel也都有一个偏置。

In [43]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

由于W的维度是4，我们需要把x也转换成4维，第二、第三维度是图片的宽度和长度，第四维度是color channel个数。

In [44]:
x_images = tf.reshape(x, [-1, 28, 28, 1])

In [46]:
h_conv1 = tf.nn.relu(conv2d(x_images, W_conv1) + b_conv1)
h_pool1 = max_pool_2_plus_2(h_conv1)

### 第二个卷积层

为了创建深度网络，我们堆积多个卷积层。

In [50]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2_plus_2(h_conv2)

### 连接层

现在，图片的大小是7*7，我们添加一个全连接层，神经元个数是1024.

In [51]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

In [53]:
a = tf.Variable(tf.zeros([7, 8]))
b = tf.Variable(tf.ones([8, 9]))

sess.run(tf.global_variables_initializer())